<a href="https://colab.research.google.com/github/verm024/bsd-capstone/blob/main/Capstone_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Code Steps**
<br>

1.   Import dataset from spreadsheet URL as csv 
2.   Split the data into training, validation, and test data
3.   Convert the data to input pipeline using tf.data
4.   Extract feature columns
5.   Create the architecture of the model
6.   Train, validate, and test the model
7.   Export the model to TFX format
8.   Deploy and test the deployed model

## Import Library

In [1]:
# Import
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

## Load Dataset

In [2]:
# Load Dataset CSV
URL = 'https://docs.google.com/spreadsheets/d/1h6TldA5tUq2Vu0wHfAzHqmtSkpKHHyL6ONOaf8F4PII/export?gid=0&format=csv'
df = pd.read_csv(URL)
df

,umur,gaji,tanggungan,pekerjaan,diterima
0,25,1.8,5,4,1
1,19,0.8,1,7,1
2,24,2.6,2,3,1
3,17,0.1,1,0,1
4,20,0.5,2,7,1
...,...,...,...,...,...
195,67,8.7,1,1,0
196,68,7.2,2,1,0
197,70,7.5,1,1,0
198,63,5.3,1,1,0


## Split Train, Validation, Test

In [3]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

128 train examples
32 validation examples
40 test examples


## Convert dataframe to tf.data


In [4]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('diterima')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [5]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)
print(train_ds)

<BatchDataset shapes: ({umur: (None,), gaji: (None,), tanggungan: (None,), pekerjaan: (None,)}, (None,)), types: ({umur: tf.int64, gaji: tf.float64, tanggungan: tf.int64, pekerjaan: tf.int64}, tf.int64)>


In [6]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['umur'])
  print('A batch of targets:', label_batch )

Every feature: ['umur', 'gaji', 'tanggungan', 'pekerjaan']
A batch of ages: tf.Tensor([47 37 44 40 44], shape=(5,), dtype=int64)
A batch of targets: tf.Tensor([1 1 1 0 1], shape=(5,), dtype=int64)


## Extract Feature Columns

In [7]:
example_batch = next(iter(train_ds))[0]

def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [8]:
umur = feature_column.numeric_column("umur")
demo(umur)
umur_buckets = feature_column.bucketized_column(umur, boundaries=[25, 34, 43, 52, 61, 70, 79])
demo(umur_buckets)

[[26.]
 [41.]
 [49.]
 [17.]
 [43.]]
[[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]]


In [9]:
# 0	Tidak Bekerja
# 1	Pensiunan
# 2	PNS
# 3	Wiraswasta
# 4	Karyawan
# 5	Petani/Nelayan/Peternak
# 6	Pegawai Pemerintah
# 7	Pekerja kasar
# 8	Buruh pabrik
# 9	Pemuka Agama
# 10	Lainnya

pekerjaan = feature_column.categorical_column_with_vocabulary_list(
      'pekerjaan', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

pekerjaan_one_hot = feature_column.indicator_column(pekerjaan)
demo(pekerjaan_one_hot)

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


In [10]:
feature_columns = []

# numeric cols
for header in ['umur', 'gaji', 'tanggungan', 'pekerjaan']:
  feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
umur_buckets = feature_column.bucketized_column(umur, boundaries=[25, 34, 43, 52, 61, 70, 79])
feature_columns.append(umur_buckets)

# categorical cols
pekerjaan = feature_column.categorical_column_with_vocabulary_list(
      'pekerjaan', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

pekerjaan_one_hot = feature_column.indicator_column(pekerjaan)
feature_columns.append(pekerjaan_one_hot)

In [11]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [12]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [13]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['umur'])
  print('A batch of targets:', label_batch )

Every feature: ['umur', 'gaji', 'tanggungan', 'pekerjaan']
A batch of ages: tf.Tensor(
[37 32 42 52 17 44 48 43 37 39 55 69 34 20 40 48 44 59 42 51 52 48 44 70
 51 37 55 42 39 43 47 31], shape=(32,), dtype=int64)
A batch of targets: tf.Tensor([1 1 0 0 1 1 0 0 0 1 1 0 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0], shape=(32,), dtype=int64)


In [14]:
tf.keras.backend.clear_session()
model = tf.keras.models.Sequential([
  feature_layer,
  layers.Dense(64, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [15]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=100)

Epoch 1/100
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1/4 [======>.......................] - ETA: 2s - loss: 0.8846 - accuracy: 0.3438WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'umur': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=int64>, 'gaji': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=float64>, 'tanggungan': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=int64>, 'pekerjaan': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=int64>}
Consider rewriting this model with the Functional API.
4/4 [==============================] - 1s 79ms/step - loss: 0.8747 - accuracy: 0.3438 - val_loss: 0.7978 - val_accuracy: 0.4375
Epoch 2/100
4/4 [==============================] - 0s 7ms/step - loss: 0.7706 - accuracy: 0.4141 - val_loss: 0.7850 - val_accuracy: 0.4375
Epoch 3/100
4/4 [==============================] - 0s 6ms/step - loss: 0.

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features_3 (DenseFeatu multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  1536      
_________________________________________________________________
dense_1 (Dense)              multiple                  4160      
_________________________________________________________________
dense_2 (Dense)              multiple                  65        
Total params: 5,761
Trainable params: 5,761
Non-trainable params: 0
_________________________________________________________________


In [17]:
loss, accuracy = model.evaluate(test_ds)
print("Test Accuracy", accuracy)

2/2 [==============================] - 0s 6ms/step - loss: 0.0890 - accuracy: 0.9750
Test Accuracy 0.9750000238418579


## Tensorflow Serving



In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update
!apt-get install tensorflow-model-server

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  81750      0 --:--:-- --:--:-- --:--:-- 81750
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:7 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 Packages [340 B]
Hit

## Save Model

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import os
import tempfile

MODEL_DIR = tempfile.gettempdir() # MODEL_DIR='/tmp'
# MODEL_DIR = "/content/drive/MyDrive/Capstone/Model"
version = 2

export_path = os.path.join(MODEL_DIR, str(version))

if os.path.isdir(export_path):
    print('\nAlready saved a model, cleaning up\n')
    !rm -r {export_path}

model.save(export_path, save_format="tf")

print('\nexport_path = {}'.format(export_path))
!ls -l {export_path}

Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
INFO:tensorflow:Assets written to: /tmp/2/assets

export_path = /tmp/2
total 256
drwxr-xr-x 2 root root   4096 Jun  8 14:31 assets
-rw-r--r-- 1 root root  13524 Jun  8 14:31 keras_metadata.pb
-rw-r--r-- 1 root root 236258 Jun  8 14:31 saved_model.pb
drwxr-xr-x 2 root root   4096 Jun  8 14:31 variables


In [19]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['gaji'] tensor_info:
        dtype: DT_DOUBLE
        shape: (-1, 1)
        name: serving_default_gaji:0
    inputs['pekerjaan'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 1)
        name: serving_default_pekerjaan:0
    inputs['tanggungan'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 1)
        name: serving_default_tanggungan:0
    inputs['umur'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 1)
        name: serving_default_umur:0
  The given S

In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR
print(MODEL_DIR)

/tmp


## Serve model

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=bsd \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 3 in a separate thread.


In [ ]:
!tail server.log

2021-05-27 11:50:58.538642: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /tmp/1
2021-05-27 11:50:58.549654: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:277] SavedModel load for tags { serve }; Status: success: OK. Took 76786 microseconds.
2021-05-27 11:50:58.551312: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /tmp/1/assets.extra/tf_serving_warmup_requests
2021-05-27 11:50:58.551471: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: bsd version: 1}
2021-05-27 11:50:58.552157: I tensorflow_serving/model_servers/server_core.cc:486] Finished adding/updating models
2021-05-27 11:50:58.552227: I tensorflow_serving/model_servers/server.cc:367] Profiler service is enabled
2021-05-27 11:50:58.552843: I tensorflow_serving/model_servers/server.cc:393] Running gRPC ModelServer at 0.0.0.0:8500 ...
[warn] g

## Sample request

In [ ]:
import json
import requests

xs = [{"gaji": [test.iloc[0][1]], "pekerjaan": [int(test.iloc[0][3])], "tanggungan": [int(test.iloc[0][2])], "umur": [int(test.iloc[0][0])]}]
print("Data:", xs)

data = json.dumps({"signature_name": "serving_default", "instances": xs})

headers = {"content-type": "application/json"}
json_response = requests.post('http://34.101.148.92:8501/v1/models/bsd:predict', data=data, headers=headers)

predictions = json.loads(json_response.text)['predictions']
print(predictions)

Data: [{'gaji': [2.2], 'pekerjaan': [5], 'tanggungan': [4], 'umur': [57]}, {'gaji': [0.0], 'pekerjaan': [0], 'tanggungan': [1], 'umur': [65]}]
[[0.833278537], [0.96526444]]
